# Project: Bollinger Bands Mean Reversion Strategy

When prices move far from their average (more than a few standard deviations), they tend to revert.
We’ll go long when the price dips below the lower Bollinger Band (oversold) and short when it rises above the upper band (overbought).

We’ll only take Bollinger Band mean reversion signals when the RSI confirms oversold or overbought conditions — improving signal quality and avoiding “catching falling knives.”

## Imports

In [110]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Setup

In [118]:
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

ticker, start_date, end_date

('AAPL', '2015-01-01', '2025-10-22')

In [119]:
df = yf.download(ticker, start=start_date, end=end_date)
df = df.droplevel(axis=1, level=1)['Close'].to_frame()
df.dropna(inplace=True)
df.head()

[*********************100%***********************]  1 of 1 completed


,Close
Date,
2015-01-02,24.261051
2015-01-05,23.577579
2015-01-06,23.579790
2015-01-07,23.910437
2015-01-08,24.829128


## Set Indicators

In [120]:
window = 20
num_std = 2
rsi_period = 14

In [121]:
df['MA'] = df['Close'].rolling(window).mean()
df['STD'] = df['Close'].rolling(window).std()
df['Upper'] = df['MA'] + num_std * df['STD']
df['Lower'] = df['MA'] - num_std * df['STD']


In [123]:
df.dropna()

,Close,MA,STD,Upper,Lower
Date,,,,,
2015-01-30,25.998573,24.552078,0.797320,26.146717,22.957438
2015-02-02,26.324778,24.655264,0.886254,26.427772,22.882756
2015-02-03,26.329214,24.792846,0.922970,26.638786,22.946905
2015-02-04,26.531147,24.940414,0.954222,26.848858,23.031969
2015-02-05,26.720518,25.080918,1.000352,27.081621,23.080215
...,...,...,...,...,...
2025-10-15,249.339996,252.674999,5.381145,263.437289,241.912710
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395


In [124]:
delta = df.Close.diff()

In [125]:
df

,Close,MA,STD,Upper,Lower
Date,,,,,
2015-01-02,24.261051,NaN,NaN,NaN,NaN
2015-01-05,23.577579,NaN,NaN,NaN,NaN
2015-01-06,23.579790,NaN,NaN,NaN,NaN
2015-01-07,23.910437,NaN,NaN,NaN,NaN
2015-01-08,24.829128,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2025-10-15,249.339996,252.674999,5.381145,263.437289,241.912710
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395


In [134]:
# RSI Calculation

delta = df.Close.diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
avg_gain = pd.Series(gain).rolling(rsi_period).mean()
avg_loss = pd.Series(loss).rolling(rsi_period).mean()
rs = avg_gain / avg_loss

df['RSI'] = 100 - (100 / (1 + rs))
#df.dropna(inplace=True)

In [139]:
100 - (100 / (1 + rs))

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
2712    35.524772
2713    34.881046
2714    46.468641
2715    59.500623
2716    59.205237
Length: 2717, dtype: float64

In [135]:
df

,Close,MA,STD,Upper,Lower,RSI
Date,,,,,,
2015-01-02,24.261051,NaN,NaN,NaN,NaN,NaN
2015-01-05,23.577579,NaN,NaN,NaN,NaN,NaN
2015-01-06,23.579790,NaN,NaN,NaN,NaN,NaN
2015-01-07,23.910437,NaN,NaN,NaN,NaN,NaN
2015-01-08,24.829128,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-10-15,249.339996,252.674999,5.381145,263.437289,241.912710,NaN
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548,NaN
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395,NaN
